In [65]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'ClusterCast.settings'
import test_utils
import django
django.setup()
import sys
sys.path.append("/home/ajp031/StockDeepLearning/ClusterCast/ClusterCast")
from asgiref.sync import sync_to_async
from importlib import reload
import ClusterPipeline.models.ClusterProcessing as cp
import ClusterPipeline.models.SequencePreprocessing as sp
import ClusterPipeline.models.TSeriesPreproccesing as tsp
from copy import deepcopy
import numpy as np
from tensorflow.keras.backend import clear_session
import random
from tensorflow.keras.optimizers import Adam
import paper_utils
import tensorflow as tf

In [66]:
from pathlib import Path
current_directory = Path.cwd()
parent_directory = current_directory.parent
sys.path.append(str(parent_directory))
# import Soft_DTW.softdtwkeras.SDTWLoss
# reload softdtwkeras.SDTWLoss
# import Soft_DTW.softdtwkeras.SDTWLoss as sdtw


In [67]:
# reload(sdtw)

In [68]:
print(tf.__version__)

2.12.0


In [69]:
reload(test_utils)

<module 'test_utils' from '/home/ajp031/ClusterCastProd/Notebooks/test_utils.py'>

In [70]:
tickers = ['aapl','f', 'spy', 'qqq', 'meta']
start_date = '2007-01-01'
end_date = '2024-01-01'
steps = 50
cluster_features = ['close', 'ema200']
target_features = ['pctChgclose+{}_target'.format(i) for i in range(1, 26) ]
interval = '1d'

In [71]:
training_features = ['pctChgclose', 'pctChgVix', 'pctChgvolume', 'hiLo','opCl', 'pctChgClHi', 'pctChgClOp',
                     "pctDiff+ema50_close", 'deriv+ema50','pctDiff+smaVol5_volume', 'macd_diff','stoch_d', 'pctDiff+ema200_close',
                     "pctDiff+bb_high_low20", 'pctDiff+bb_high_close20', 'pctDiff+bb_low_close20',
                     'pctDiff+ema10_close', 'pctDiff+ema20_close', 'pctDiff+ema100_close',
                     'deriv+ema10', 'deriv+ema20','deriv+ema100']

# training_features += ['open', 'high', 'low', 'close','volume', 'Vix', 'ema10', 'ema20','ema50','ema100']

In [72]:
scaling_dict = {
    "price_vars": sp.ScalingMethod.SBSG,
    "trend_vars": sp.ScalingMethod.SBSG,
    "pctChg_vars": sp.ScalingMethod.STANDARD,
    "rolling_vars": sp.ScalingMethod.STANDARD,
    "target_vars": sp.ScalingMethod.QUANT_MINMAX,
    "lag_feature_vars": sp.ScalingMethod.STANDARD,
    "momentum_vars": sp.ScalingMethod.STANDARD,
}

In [73]:
group_params = cp.StockClusterGroupParams(
    tickers=tickers,
    start_date=start_date,
    end_date=end_date,
    n_steps=steps,
    cluster_features=cluster_features,
    target_cols=target_features,
    interval=interval,
    training_features=training_features,
)

In [74]:
group_params.set_scaling_dict(scaling_dict)
group_params.initialize() 

In [75]:
data_sets = []
for ticker in tickers:
    data_set = tsp.StockDataSet(group_params, ticker)
    data_set.preprocess_pipeline(to_train = True)
    data_sets.append(data_set)

group_params = data_sets[0].group_params

group_params.data_sets = data_sets    


Creating and Processing Dataset
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Empty DataFrame
Columns: []
Index: [2007-01-03 00:00:00]
Momentum Features Created
Scaling Standard
Scaling Quant Min Max Features
Quant Min Max Features Scaled
RandomForest Compete
Dataset Preprocessing Complete
Creating and Processing Dataset
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Empty DataFrame
Columns: []
Index: [2007-01-03 00:00:00]
Momentum Features Created
Scaling Standard
Scaling Quant Min Max Features
Quant Min Max Features Scaled
RandomForest Compete
Dataset Preprocessing Complete
Creating and Processing Dataset
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Empty DataFrame
Columns: []
Index: [2007-01-03 00:00:00]
Momentu

KeyboardInterrupt: 

: 

In [ ]:
data_set = group_params.data_sets[0]

In [ ]:
training_df = deepcopy(data_set.training_df )
test_df = deepcopy(data_set.test_df) 

In [ ]:
training_df.columns

In [ ]:
training_df = training_df[training_features + target_features]
test_df = test_df[training_features + target_features]

In [ ]:
training_df.describe()

In [ ]:
training_features

In [ ]:
# data visualizations 
import matplotlib.pyplot as plt
import seaborn as sns

# sns.pairplot(training_df[training_features])

In [ ]:
# correlation matrix
corr = training_df[training_features].corr()
sns.heatmap(corr, annot=False)


In [ ]:
# hist plots
training_df[training_features].hist(bins=50, figsize=(20,15))

In [ ]:
# line plots 
training_df[training_features].plot(subplots=True, figsize=(20,15))

In [ ]:
# box plots in seaborn
# for feature in training_features:
#     sns.boxplot(x=feature, data=training_df)
#     plt.show()

In [ ]:
sequence_set = sp.StockSequenceSet(group_params)

In [ ]:
sequence_set.preprocess_pipeline(add_cuma_pctChg_features = False)

In [ ]:
training_seq_elements = group_params.train_seq_elements
test_seq_elements = group_params.test_seq_elements

In [ ]:
X_train, y_train = sp.SequenceElement.create_array(training_seq_elements)
X_test, y_test = sp.SequenceElement.create_array(test_seq_elements)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
X_feature_dict = group_params.X_feature_dict
y_feature_dict = group_params.y_feature_dict

In [ ]:
X_train_filtered = paper_utils.filter_by_features(X_train,training_features, X_feature_dict)
X_test_filtered = paper_utils.filter_by_features(X_test,training_features, X_feature_dict)
y_train_filtered = paper_utils.filter_y_by_features(y_train,target_features, y_feature_dict)
y_test_filtered = paper_utils.filter_y_by_features(y_test,target_features, y_feature_dict)

In [ ]:
reload(paper_utils)

In [ ]:
print(X_train_filtered.shape, y_train_filtered.shape)
print(X_test_filtered.shape, y_test_filtered.shape)

In [ ]:
# expand y dims 
y_train_filtered = np.expand_dims(y_train_filtered, axis=-1)
y_test_filtered = np.expand_dims(y_test_filtered, axis=-1)

In [ ]:
X_cluster_train = paper_utils.filter_by_features(X_train,cluster_features, X_feature_dict)
X_cluster_test = paper_utils.filter_by_features(X_test,cluster_features, X_feature_dict)

In [ ]:
from tslearn.clustering import TimeSeriesKMeans

n_clusters = 4
km = TimeSeriesKMeans(n_clusters=n_clusters, verbose=True, random_state=42, metric = 'euclidean')
km.fit(X_cluster_train)
train_labels = km.labels_ 
test_labels = km.predict(X_cluster_test)

clusters = []
for i in range(n_clusters):
    print('Cluster {} has {} samples'.format(i, np.sum(train_labels == i)))
    X_train_clustered = X_train_filtered[train_labels == i]
    y_train_clustered = y_train_filtered[train_labels == i]
    X_test_clustered = X_test_filtered[test_labels == i]
    y_test_clustered = y_test_filtered[test_labels == i]

    cur_train_seq_elements = [] 
    cur_test_seq_elements =[ ]
    for j in range(len(test_seq_elements)):
        if train_labels[j] == i:
            cur_train_seq_elements.append(training_seq_elements[j])
    for j in range(len(test_seq_elements)):
        if test_labels[j] == i:
            cur_test_seq_elements.append(test_seq_elements[j])

    cluster = paper_utils.Cluster(i, X_train_clustered, y_train_clustered, X_test_clustered, y_test_clustered, cur_train_seq_elements, cur_test_seq_elements)
    clusters.append(cluster)
    


In [ ]:
for i in range(n_clusters):
    test_utils.visualize_cluster_centroid(km.cluster_centers_[i], cluster_features)
    plt.show()

In [ ]:
reload(paper_utils)

In [ ]:
total_cluster = paper_utils.Cluster(-1, X_train_filtered, y_train_filtered, X_test_filtered, y_test_filtered, training_seq_elements, test_seq_elements)

In [ ]:
import test_utils_models
reload(test_utils_models)

In [ ]:
baseline_model = test_utils_models.CustomSeq2SeqModel(len(target_features), X_train_filtered.shape[1], X_train_filtered.shape[2])

In [ ]:
lr = 0.00001
batch_size = 128
epochs = 250
# loss = sdtw.SDTWLoss(gamma = 1.0)
# loss = test_utils_models.dtw_loss
loss = 'mse'

baseline_model.compile(optimizer = Adam(learning_rate=lr), loss = loss)

In [ ]:
print(X_train_filtered.shape)
print(y_train_filtered.shape)
print(X_test_filtered.shape)
print(y_test_filtered.shape)

In [ ]:
# y_true_dummy = tf.random.normal([10, 25, 1])  # Adjust dimensions as per actual data
# y_pred_dummy = tf.random.normal([10, 25, 1])

# result = loss.batch_squared_euclidean_compute_tf(y_true_dummy, y_pred_dummy)
# print(result)
# reload(sdtw)

In [ ]:
total_cluster.train_model(baseline_model, epochs, batch_size, shuffle = True)

In [ ]:
total_cluster = paper_utils.Cluster(-1, X_train_filtered, y_train_filtered, X_test_filtered, y_test_filtered, training_seq_elements, test_seq_elements)
total_cluster.model = baseline_model

In [ ]:
baseline_output_string, baseline_results, attention_weights = total_cluster.eval_model_ragged()

In [ ]:
baseline_output_string, baseline_results, attention_weights = total_cluster.eval_model()

In [ ]:
print(baseline_output_string)

In [ ]:
test_utils.visualize_future_distribution(baseline_results)

In [ ]:
# freeze leyers
baseline_model.freeze_encoder()

In [ ]:
# save weights 
baseline_model.save_weights('baseline_model.weights.h5')

In [ ]:
lr = 0.0001
batch_size = 128
epochs = 150
loss = test_utils_models.dtw_loss
# loss = 'mae'

In [ ]:
for i in range(n_clusters):
    print('Cluster {}'.format(i))
    cluster = clusters[i]
    model = test_utils_models.CustomSeq2SeqModel(len(target_features), X_train_filtered.shape[1], X_train_filtered.shape[2])
    model.build((None,X_train_filtered.shape[1], X_train_filtered.shape[2]))
    model.load_weights('baseline_model.weights.h5')
    model.freeze_encoder()
    model.compile(optimizer = Adam(learning_rate=lr), loss = loss)
    cluster.train_model(model, epochs, batch_size)
    output_string, results, attention_weights = cluster.eval_model()
    print(output_string)

In [ ]:
# loop through the clusters and access model_result_dict where model_result_dict[day] has keys
# total_predictions, correct_predictions, 'accuracy', 'mse', mae. We need to compute a total weighted accuracy,
# mse, and mae for all clusters. for each day in model_result_dict

model_result_dict_clusters = {}

for day in range(1, (y_test_filtered.shape[1]+1)):
    total_predictions = []
    mse = []
    mae = []
    correct_predictions = []
    cum_mae = []
    cum_mse = []

    for cluster in clusters:
        # Append necessary metrics
        cluster_data = cluster.model_result_dict[str(day)]
        
        total_predictions.append(cluster_data['total_predictions'])
        correct_predictions.append(cluster_data['correct_predictions'])
        mse.append(cluster_data['mse'])
        mae.append(cluster_data['mae'])
        cum_mae.append(cluster_data['cum_mae'])
        cum_mse.append(cluster_data['cum_mse'])

    # Compute weighted MSE and MAE
    total_all_predictions = sum(total_predictions)

    weighted_mse = sum(mse[i] * total_predictions[i] for i in range(len(mse))) / total_all_predictions
    weighted_mae = sum(mae[i] * total_predictions[i] for i in range(len(mae))) / total_all_predictions
    weighted_cum_mae = sum(cum_mae[i] * total_predictions[i] for i in range(len(cum_mae))) / total_all_predictions
    weighted_cum_mse = sum(cum_mse[i] * total_predictions[i] for i in range(len(cum_mse))) / total_all_predictions

    # Store results in dictionary
    if day not in model_result_dict_clusters:
        model_result_dict_clusters[day] = {}
    model_result_dict_clusters[day]['mse'] = weighted_mse
    model_result_dict_clusters[day]['mae'] = weighted_mae
    model_result_dict_clusters[day]['total_predictions'] = total_all_predictions
    model_result_dict_clusters[day]['correct_predictions'] = sum(correct_predictions)
    model_result_dict_clusters[day]['cum_mae'] = weighted_cum_mae
    model_result_dict_clusters[day]['cum_mse'] = weighted_cum_mse

    model_result_dict_clusters[day]['accuracy'] = sum(correct_predictions) / total_all_predictions * 100 

# You can then access these results as needed

    

In [ ]:
paper_utils.plot_results(total_cluster.model_result_dict, model_result_dict_clusters)

In [ ]:
for cluster in clusters:
    output_string, results, attention_weights = cluster.eval_model()
    fig = test_utils.visualize_future_distribution(results)
    fig.show()

In [ ]:
y_feature_sets = group_params.y_feature_sets
total_cluster.inverse_transform(y_feature_sets, target_features)

In [ ]:
for cluster in clusters:
    cluster.inverse_transform(y_feature_sets, target_features)

In [ ]:
target_feature = 'pctChgclose+1_target'
scaler = [feature_set for feature_set in y_feature_sets if feature_set.name == target_feature][0].scaler

In [ ]:
for i in range(0,20):
    print('Sequence Element {}'.format(i))
    
    seq_element = test_seq_elements[i]
    baseline_prediction,baseline_attention_weights = total_cluster.get_next_prediction_value(seq_element.start_date)

    cluster_label = test_labels[i]
    cluster = clusters[cluster_label]
    cluster_prediction, cluster_attention_weights = cluster.get_next_prediction_value(seq_element.start_date)

    print('baseline weights')
    test_utils.plot_attention_weights_single(baseline_attention_weights)

    print('cluster weights label {}'.format(cluster_label))
    test_utils.plot_attention_weights_single(cluster_attention_weights)

    paper_utils.visualize_future_movement_mpl(seq_element, baseline_prediction, cluster_prediction, scaler, target_features = target_features, num_days = len(target_features))



In [ ]:
reload(paper_utils)
reload(test_utils)

In [ ]:
print(len(clusters[0].test_seq_elements))
print(clusters[0].X_test.shape)